In [ ]:
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler
import time 
import os
import copy

In [ ]:
train_data = np.load("/content/train_data_4.npy", allow_pickle=True)
val_data = np.load("/content/val_data_4.npy", allow_pickle=True)

In [ ]:
val_data.shape

(56, 2)

In [ ]:
class Data(Dataset):
    def __init__(self, datafile):
        self.datafile = datafile

    def __len__(self):
        return self.datafile.shape[0]

    def __getitem__(self, idx):
        img = self.datafile[idx][0]
        y = int(self.datafile[idx][1])
        if y == 0:
            lbl = torch.tensor([1.0, 0.0])
        if y == 1:
            lbl = torch.tensor([0.0, 1.0])
        
        return img, lbl

In [ ]:
# Make data class
train_data_class = Data(train_data)
valid_data_class = Data(val_data)

In [ ]:
# Make data loader
train_loader = DataLoader(train_data_class, batch_size= 16, shuffle=True)
val_loader = DataLoader(valid_data_class, batch_size=16, shuffle=True)

In [ ]:
dataloaders = {"train" : train_loader, "val" : val_loader}
dataset_sizes = {"train":train_data.shape[0], "val" : val_data.shape[0]}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# !pip install facenet_pytorch

In [ ]:
from facenet_pytorch import InceptionResnetV1

In [ ]:
my_net = InceptionResnetV1(pretrained='vggface2',classify=True).eval()

In [ ]:
for param in my_net.parameters():
    param.requires_grad = False

In [ ]:
my_net.logits = nn.Sequential(nn.Linear(512, 2), torch.nn.Sigmoid())

In [ ]:
# my_net.to(device)

In [ ]:
criterion = nn.MSELoss()

optimizer = optim.Adam(my_net.parameters(), lr=0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

To do with resnet

In [ ]:
# ## Model as feature extractor
# model_conv = models.resnet18(pretrained=True)
# for param in model_conv.parameters():
#     param.requires_grad = False

# # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Sequential(nn.Linear(num_ftrs, 2), torch.nn.Sigmoid())

# criterion = nn.MSELoss()

# # Observe that only parameters of final layer are being optimized as opposed to before.
# #optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)
# optimizer_conv = optim.Adam(model_conv.fc.parameters(), lr=0.001)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
# Testing with one batch

img, lbl = next(iter(dataloaders['val']))

outputs = my_net(img)
value, preds = torch.max(outputs, 1)
#lbl = lbl.squeeze_()
loss = criterion(outputs, lbl)

print(loss)

tensor(0.2605, grad_fn=<MseLossBackward>)


In [ ]:
my_net.to(device)

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [ ]:
def train_model(model, optimizer,criterion, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 1.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    value, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                
                running_corrects += torch.sum(preds == torch.argmax(labels,1)) #labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_conv = train_model(my_net,  optimizer_conv, criterion,  exp_lr_scheduler,
                       num_epochs=51)

Epoch 0/50
----------


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.2437 Acc: 0.6144
val Loss: 0.2148 Acc: 0.7143

Epoch 1/50
----------
train Loss: 0.2448 Acc: 0.5932
val Loss: 0.2160 Acc: 0.6964

Epoch 2/50
----------
train Loss: 0.2446 Acc: 0.5975
val Loss: 0.2169 Acc: 0.7321

Epoch 3/50
----------
train Loss: 0.2501 Acc: 0.5847
val Loss: 0.2130 Acc: 0.6964

Epoch 4/50
----------
train Loss: 0.2522 Acc: 0.5678
val Loss: 0.2120 Acc: 0.7321

Epoch 5/50
----------
train Loss: 0.2503 Acc: 0.5763
val Loss: 0.2229 Acc: 0.6964

Epoch 6/50
----------
train Loss: 0.2489 Acc: 0.6186
val Loss: 0.2134 Acc: 0.7143

Epoch 7/50
----------
train Loss: 0.2497 Acc: 0.5890
val Loss: 0.2157 Acc: 0.7321

Epoch 8/50
----------
train Loss: 0.2451 Acc: 0.6017
val Loss: 0.2153 Acc: 0.7321

Epoch 9/50
----------
train Loss: 0.2503 Acc: 0.5890
val Loss: 0.2185 Acc: 0.7500

Epoch 10/50
----------
train Loss: 0.2554 Acc: 0.5636
val Loss: 0.2253 Acc: 0.8036

Epoch 11/50
----------
train Loss: 0.2505 Acc: 0.5763
val Loss: 0.2157 Acc: 0.7321

Epoch 12/50
----------
t

In [ ]:
torch.save(model_conv.state_dict(), "/content/model_Inc.pth")